<a href="https://colab.research.google.com/github/FernandoSNunes/AR-Educational-Framework/blob/master/amd_CUDA.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
# importa macro %%cu
!pip install git+git://github.com/andreinechaev/nvcc4jupyter.git

# carrega plugin
%load_ext nvcc_plugin

  Cloning git://github.com/andreinechaev/nvcc4jupyter.git to /tmp/pip-req-build-09wzlf39
  Running command git clone -q git://github.com/andreinechaev/nvcc4jupyter.git /tmp/pip-req-build-09wzlf39
  Created wheel for NVCCPlugin: filename=NVCCPlugin-0.0.2-cp36-none-any.whl size=4308 sha256=66ea66f1ba3ee6446d3db35b71f44e2d8db22ac1d326d079157aa76b5ac675ec
  Stored in directory: /tmp/pip-ephem-wheel-cache-teld3wzg/wheels/10/c2/05/ca241da37bff77d60d31a9174f988109c61ba989e4d4650516
Successfully built NVCCPlugin
The nvcc_plugin extension is already loaded. To reload it, use:
  %reload_ext nvcc_plugin


In [17]:
%%cu

#include <stdio.h>
#include <stdlib.h>
#include <stdint.h>
#include <string.h>
#include <pthread.h>
#include <math.h>

#define MAX_BLOCK 128
#define MAX_THREADS 32
#define CORE 4

//First line: v (number of vertices) and e (number of edges)
uint32_t v, e;
uint32_t* dists;

uint32_t *smd; 	//sum of minimum distances
uint32_t *paths; //number of paths found


// Solves the minimum distance between all pairs of vertices

__global__
void md_all_pairs (uint32_t* dists, uint32_t v) {
    //int _id = (int )(id);
    //long int _id = (long int)(id);
    uint32_t k, i, j;

    uint32_t portion_size = v/(blockDim.x * gridDim.x);
    uint32_t i_real = (blockIdx.x * blockDim.x + threadIdx.x)/portion_size;



    //if (v%(blockDim.x * gridDim.x) != 0)
    //    portion_size++;


    printf(" _id = %d, v = %d", i_real, v);
    for (i = i_real*portion_size; (i < ((i_real+1)* portion_size)) && (i < v); ++i) {
        printf ("thread %ld bloco %ld calcula i = %d\n",threadIdx.x ,blockIdx.x, i);
        for (k = 0; k < v; ++k) {
            for (j = 0; j < v; ++j) {           
                uint32_t intermediary = dists[i*v+k] + dists[k*v+j];
                // Checks for overflows
                if ((intermediary >= dists[i*v+k])&&(intermediary >= dists[k*v+j])&&(intermediary < dists[i*v+j]))
                    dists[i*v+j] = dists[i*v+k] + dists[k*v+j];
            }
        }
    }

}

__global__
void amd_threads (uint32_t* dists, uint32_t v, uint32_t *smd, uint32_t *paths) 
{    
    uint32_t i, j;
	  uint32_t infinity = v*v;
    
    uint32_t portion_size = v/(blockDim.x * gridDim.x);
    uint32_t i_real = (blockIdx.x * blockDim.x + threadIdx.x)/portion_size;

    
    //if (v%(blockDim.x * gridDim.x) != 0)        //divisao nao exata
    //    portion_size++;

    for (i = i_real*portion_size; (i < ((i_real+1)* portion_size)) && (i < v); ++i) {
        for (j = 0; j < v; ++j) {
			// We only consider if the vertices are different and there is a path
            if ((i != j) && (dists[i*v+j] < infinity)) {
				smd[i_real] += dists[i*v+j];       //salva 
				paths[i_real]++;
			}
        }
    }
}

/* Computes the average minimum distance between all pairs of vertices with a path connecting them */
void amd (uint32_t* dists, uint32_t v, uint32_t div_t, uint32_t div_b) {
    uint32_t *smd;
    uint32_t *paths;

    cudaMallocManaged(&smd, (div_t * div_b) * sizeof(uint32_t));
    cudaMallocManaged(&paths, (div_t * div_b) * sizeof(uint32_t));
    
    for (int i = 0; i < (div_t * div_b); i++){
        smd[i] = 0;
        paths[i] = 0;
    }

	uint32_t solution = 0, smd_sum = 0, paths_sum = 0;
    
    amd_threads<<<div_b, div_t>>>(dists, v, smd, paths);

	  cudaDeviceSynchronize();

    for (int i = 0; i < (div_t * div_b); i++){
        smd_sum += smd[i];
        paths_sum += paths[i];
    }

	solution = smd_sum / paths_sum;
	printf("%d\n", solution);
}

/* Debug function (not to be used when measuring performance)*/
void debug (uint32_t* dists, uint32_t v) {
    uint32_t i, j;
	uint32_t infinity = v*v;
    

    for (i = 0; i < v; ++i) {
        for (j = 0; j < v; ++j) {
            if (dists[i*v+j] > infinity) printf("%7s", "inf");
            else printf ("%7u", dists[i*v+j]);
        }
        printf("\n");
    }
}

// Main program - reads input, calls FW, shows output
int main (int argc, char* argv[]) {

    //Reads input 
    scanf("%u %u", &v, &e);

    uint32_t *dists;

    cudaMallocManaged(&dists, v*v*sizeof(uint32_t));

    //Allocates distances matrix (w/ size v*v) i
    //and sets it with max distance and 0 for own vertex
    memset(dists, UINT32_MAX, v*v*sizeof(uint32_t));
    uint32_t i;
    for ( i = 0; i < v; ++i ) dists[i*v+i] = 0;

    //Reads edges from file and sets them in the distance matrix
    uint32_t source, dest, cost;
    for ( i = 0; i < e; ++i ){
        scanf("%u %u %u", &source, &dest, &cost);
        if (cost < dists[source*v+dest]) dists[source*v+dest] = cost;
    }


   //calcula a divisao para enviar pra gpu
   uint32_t div_t = v*v, div_b = 1;
   while (div_t > MAX_THREADS){   //tem que ficar abaixo do limite de threads
       div_t /= 2;
       div_b *= 2;
   }
   if (div_t != MAX_THREADS){   //divisao inexata de linhas
       div_t = MAX_THREADS;
   }
   if (div_b > MAX_BLOCK){
      div_b = MAX_BLOCK;
   }

  md_all_pairs<<<div_b, div_t>>>(dists, v);

	// espera as threads acabarem
  cudaDeviceSynchronize();



	//Computes the minimum distance for all pairs of vertices
    //md_all_pairs(dists, v);
    //Computes and prints the final solution
    amd(dists, v, div_t, div_b);


	// Free memory
  cudaFree(dists);


#if DEBUG
	debug(dists, v);
#endif

    return 0;
}

 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0 _id = -1, v = 0
